# Импорт библиотек

In [5]:
import os
import warnings
import logging
import shutil
import json
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from ultralytics import YOLO
from segment_anything import sam_model_registry, SamPredictor
import torch


# Настройки


In [6]:
warnings.filterwarnings('ignore')
logging.basicConfig(level=logging.INFO)


# Константы


In [ ]:
CLASSES = {'fire': 0, 'smoke': 1}
TARGET_METRICS = {
    'fire': {'precision': 0.7, 'recall': 0.6, 'mAP50': 0.6},
    'smoke': {'precision': 0.8, 'recall': 0.8, 'mAP50': 0.8}
}

# 1. Подготовка данных
def prepare_dataset():
    # Очистка предыдущих данных
    shutil.rmtree('yolo_dataset', ignore_errors=True)
    
    # Создание структуры папок
    os.makedirs('yolo_dataset/images/train', exist_ok=True)
    os.makedirs('yolo_dataset/images/val', exist_ok=True)
    os.makedirs('yolo_dataset/labels/train', exist_ok=True)
    os.makedirs('yolo_dataset/labels/val', exist_ok=True)

    # Функция конвертации COCO в YOLO формат
    def convert_coco_to_yolo(json_path, img_dir, out_img_dir, out_label_dir):
        with open(json_path) as f:
            data = json.load(f)
        
        images = {img['id']: img['file_name'] for img in data['images']}
        
        for ann in data['annotations']:
            if ann['category_id'] not in CLASSES.values():
                continue
                
            img_name = images[ann['image_id']]
            img_info = next(img for img in data['images'] if img['id'] == ann['image_id'])
            
            # Конвертация bbox
            x, y, w, h = ann['bbox']
            x_center = (x + w/2) / img_info['width']
            y_center = (y + h/2) / img_info['height']
            width = w / img_info['width']
            height = h / img_info['height']
            
            # Сохранение аннотации
            base_name = os.path.basename(img_name)
            label_path = os.path.join(out_label_dir, os.path.splitext(base_name)[0] + '.txt')
            with open(label_path, 'a') as f:
                f.write(f"{ann['category_id']} {x_center} {y_center} {width} {height}\n")
            
            # Копирование изображения
            src = os.path.join(img_dir, img_name)
            dst = os.path.join(out_img_dir, base_name)
            if not os.path.exists(dst):
                shutil.copy(src, dst)

    # Конвертация данных
    convert_coco_to_yolo('475_fire_train/annotations/instances_default.json',
                        '475_fire_train/images',
                        'yolo_dataset/images/train',
                        'yolo_dataset/labels/train')
    
    convert_coco_to_yolo('474_fire_val/annotations/instances_default.json',
                        '474_fire_val/images',
                        'yolo_dataset/images/val',
                        'yolo_dataset/labels/val')

    # Создание YAML файла
    yaml_content = f"""
    path: {Path('yolo_dataset').absolute()}
    train: images/train
    val: images/val
    names: {CLASSES}
    """
    with open('fire_smoke.yaml', 'w') as f:
        f.write(yaml_content.strip())

# 2. Обучение модели
def train_model():
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = YOLO('yolov8x-seg.pt')  # Для сегментации
    
    results = model.train(
        data='fire_smoke.yaml',
        epochs=100,
        batch=16,
        imgsz=640,
        device=device,
        augment=True,
        hsv_h=0.015,
        hsv_s=0.7,
        hsv_v=0.4,
        degrees=15,
        flipud=0.1,
        fliplr=0.5
    )
    
    return model

# 3. Сегментация с SAM
class SAM_Enhancer:
    def __init__(self, model_type='vit_h', checkpoint='sam_vit_h_4b8939.pth'):
        self.sam = sam_model_registry[model_type](checkpoint=checkpoint)
        self.sam.to('cuda' if torch.cuda.is_available() else 'cpu')
        self.predictor = SamPredictor(self.sam)
    
    def enhance(self, image_path, yolo_model):
        # Детекция с YOLO
        yolo_results = yolo_model.predict(image_path)
        boxes = yolo_results[0].boxes.xywh.cpu().numpy()
        
        # Подготовка изображения
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        self.predictor.set_image(image)
        
        # Сегментация с SAM
        masks, _, _ = self.predictor.predict(
            point_coords=None,
            point_labels=None,
            box=boxes,
            multimask_output=False,
        )
        return masks[0]  # Возвращаем лучшую маску

# 4. Визуализация и оценка
def evaluate_model(model, sam_enhancer=None):
    # Валидация
    metrics = model.val()
    
    # Вывод метрик
    print("\nОценка модели:")
    print("{:<10} {:<10} {:<10} {:<10}".format("Объект", "Precision", "Recall", "mAP50"))
    for i, name in enumerate(CLASSES.keys()):
        print("{:<10} {:<10.2f} {:<10.2f} {:<10.2f}".format(
            name,
            metrics.box.p[i],
            metrics.box.r[i],
            metrics.box.map50[i]
        ))
    
    # Визуализация
    test_images = [f for f in os.listdir('yolo_dataset/images/val') if f.endswith(('.jpg', '.png'))][:3]
    for img_name in test_images:
        img_path = os.path.join('yolo_dataset/images/val', img_name)
        
        # Детекция YOLO
        results = model.predict(img_path)
        plt.figure(figsize=(15, 5))
        
        # Визуализация детекции
        plt.subplot(1, 2, 1)
        res_plotted = results[0].plot()
        plt.imshow(cv2.cvtColor(res_plotted, cv2.COLOR_BGR2RGB))
        plt.title("YOLOv8 Detection")
        plt.axis('off')
        
        # Визуализация сегментации (если SAM используется)
        if sam_enhancer:
            plt.subplot(1, 2, 2)
            mask = sam_enhancer.enhance(img_path, model)
            plt.imshow(mask, cmap='jet', alpha=0.5)
            plt.title("SAM Enhanced Segmentation")
            plt.axis('off')
        
        plt.tight_layout()
        plt.show()

# Основной процесс
if __name__ == "__main__":
    # 1. Подготовка данных
    prepare_dataset()
    
    # 2. Обучение модели
    model = train_model()
    
    # 3. Инициализация SAM (опционально)
    sam_enhancer = SAM_Enhancer() if input("Использовать SAM для улучшения сегментации? (y/n): ").lower() == 'y' else None
    
    # 4. Оценка модели
    evaluate_model(model, sam_enhancer)
    
    # 5. Сохранение модели
    model.save('best_fire_smoke.pt')
    model.export(format='onnx')
    
    print("\nОбучение завершено! Модель сохранена как 'best_fire_smoke.pt'")